# Retrogate Analysis

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-symmetry.ipynb

import pymongo
localMongo = pymongo.MongoClient("mongodb://localhost:27017/")
retroDb = localMongo["retro-database"]
retroCol = retroDb["retro-collection"]

In [ ]:
startBoard = ((' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '))
boards = []

def generateBoards(board=startBoard):
    global boards
    white = countStones(('', board), 'w')
    black = countStones(('', board), 'b')
    if (white == 3 and black == 3):
        boards += tuple([board])
        return
    for i in range(0,3):
        for j in range(0,8):
            if board[i][j] == ' ':
                if white == 3:
                    newBoard = place(board, (i, j), 'b')
                    generateBoards(newBoard)
                elif black == 3:
                    newBoard = place(board, (i, j), 'w')
                    generateBoards(newBoard)
                else:
                    for turn in ('w', 'b'):
                        newBoard = place(board, (i, j), turn)
                        generateBoards(newBoard)

In [ ]:
def getSymmetryUnique():
    global boards
    doubleBoards = set()
    uniqueStates = []
    stones = tuple([0, 0])
    for board in boards:
        state = tuple([stones, board])
        if not (state in doubleBoards):
            doubleBoards |= findSymmetries(('', board))
            uniqueStates.append(state)
    return uniqueStates

In [ ]:
def fillDb(states):
    fullStep = []
    stepCount = 0
    
    stepCount += 1
    _states = states
    for state in _states:
        _nextStates = nextStates(state, 'w')
        for ns in _nextStates:
            if (finished(ns, 'w') and utility(ns, 'w') == 1):
                entry = { "state": state, "nextState": ns, "steps": stepCount }
                retroCol.insert_one(entry)
                fullStep.append(state)
                states.remove(state)
                break
    
    while len(states) > 0:
        print(len(fullStep))
        _states = states
        halfStep = []
        stepCount += 1
        for state in _states:
            _nextStates = nextStates(state, 'b')
            for ns in _nextStates:
                if (all(ns in fullStep)):
                    halfStep.append(state)
                    states.remove(state)
                    break
        if len(halfStep) == 0:
            break
        
        print(len(halfStep))
        fullStep = []
        stepCount += 1
        for state in _states:
            _nextStates = nextStates(state, 'w')
            for ns in _nextStates:
                if (ns in halfStep):
                    entry = { "state": state, "nextState": ns, "steps": stepCount }
                    retroCol.insert_one(entry)
                    fullStep.append(state)
                    states.remove(state)
                    break
        if len(fullStep) == 0:
            break

In [ ]:
generateBoards()
print(len(boards))
states = getSymmetryUnique()
print(len(states))
fillDb(states)